In [2]:
import numpy as np
import json
import pandas as pd

with open("processed_data/sports.json","r") as f:
    sports = json.load(f)

with open("processed_data/finalData.json","r") as f:
    Data = json.load(f)

with open("processed_data/N0.json","r") as f:
    N0 = json.load(f)



In [3]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
import xgboost as xgb
import lightgbm as lgb
from sklearn.kernel_ridge import KernelRidge

def LinearModel(X,y):
    # 创建回归模型
    model = LinearRegression()

    # 拟合模型
    model.fit(X, y)

    # 输出回归系数
    print("截距项 (β0):", model.intercept_)
    print("回归系数 (β1, β2, β3):", model.coef_)

    # 使用模型进行预测
    y_pred = model.predict(X)
    print("预测结果:", y_pred)  
    return model

In [ ]:
from sklearn.linear_model import LogisticRegression
def LogisticModel(X,y):
    model = LogisticRegression()
    model.fit(X, y)
    return model

In [4]:

def rmsle_cv(model,x,y,n_folds = 5):
    kf = KFold(n_folds, shuffle=True, random_state=42)
    rmse= cross_val_score(model, x, y, cv = kf.get_n_splits())
    return(rmse)

# # 尝试多个模型
# model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468,
#                              learning_rate=0.05, max_depth=3,
#                              min_child_weight=1.7817, n_estimators=2200,
#                              reg_alpha=0.4640, reg_lambda=0.8571,
#                              subsample=0.5213, silent=1,
#                               nthread = -1)
# score_xgb = rmsle_cv(model_xgb)
# print("Xgboost score: {:.4f} ({:.4f})\n".format(score_xgb.mean(), score_xgb.std()))

# model_xgb.fit(x_train.values, train_labels)
# model_xgb_prec = model_xgb.predict(x_train.values)
# print(valuation(model_xgb_prec, train_labels))

# model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
#                               learning_rate=0.05, n_estimators=720,
#                               max_bin = 55, bagging_fraction = 0.8,
#                               bagging_freq = 5, feature_fraction = 0.2319,
#                               feature_fraction_seed=9, bagging_seed=9,
#                               min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
# score_lgb = rmsle_cv(model_lgb)
# print("Lightgbm score: {:.4f} ({:.4f})\n".format(score_lgb.mean(), score_lgb.std()))

# model_lgb.fit(x_train.values, train_labels)
# model_lgb_prec = model_lgb.predict(x_train.values)
# print(valuation(model_lgb_prec, train_labels))

# KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
# score_krr = rmsle_cv(KRR)
# print("KRR score: {:.4f} ({:.4f})\n".format(score_krr.mean(), score_krr.std()))


# KRR.fit(x_train.values, train_labels)
# KRR_prec = KRR.predict(x_train.values)
# print(valuation(KRR_prec, train_labels))


In [5]:
# ... existing imports ...
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

def random_search_params(x_train, train_labels, model_type='xgb', n_iter=100):
    """
    使用随机搜索方法对XGBoost或LightGBM进行参数优化
    """
    if model_type == 'xgb':
        model = xgb.XGBRegressor(nthread=-1)
        param_dist = {
            'colsample_bytree': uniform(0.3, 0.7),
            'gamma': uniform(0, 0.5),
            'learning_rate': uniform(0.01, 0.1),
            'max_depth': randint(2, 6),
            'min_child_weight': uniform(1, 3),
            'n_estimators': randint(1000, 3000),
            'reg_alpha': uniform(0, 1),
            'reg_lambda': uniform(0, 1),
            'subsample': uniform(0.5, 0.9)
        }
    else:  # lightgbm
        model = lgb.LGBMRegressor()
        param_dist = {
            'num_leaves': randint(3, 10),
            'learning_rate': uniform(0.01, 0.1),
            'n_estimators': randint(500, 1500),
            'max_bin': randint(30, 100),
            'bagging_fraction': uniform(0.6, 0.9),
            'bagging_freq': randint(2, 10),
            'feature_fraction': uniform(0.1, 0.5),
            'min_data_in_leaf': randint(3, 12),
            'min_sum_hessian_in_leaf': randint(5, 20)
        }

    random_search = RandomizedSearchCV(
        model, 
        param_distributions=param_dist,
        n_iter=n_iter,
        scoring='neg_mean_squared_error',
        cv=5,
        verbose=1,
        random_state=42,
        n_jobs=-1
    )
    
    random_search.fit(x_train, train_labels)
    # print(f"Best parameters: {random_search.best_params_}")
    # print(f"Best score: {-random_search.best_score_:.4f}")
    return random_search.best_estimator_

# model_xgb = random_search_params(x_train, train_labels, model_type='xgb')

# model_lgb = random_search_params(x_train, train_labels, model_type='lgb')

In [6]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

def GaussianProcessRegressorModel(X,y):
    kernel = C(1.0, (1e-4, 1e1)) * RBF(1.0, (1e-4, 1e1))    

    gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
    gpr.fit(X, y)
    y_pred, sigma = gpr.predict(X, return_std=True)
    print(y_pred,sigma)
    return gpr

for sport in sports:
    if len(Data[sport]) == 0:
        continue
    simple = np.array(Data[sport])
    X = np.array(simple[:,0:3])
    y = np.array(simple[:,3])
    gpr_model = GaussianProcessRegressorModel(X,y)
    print("start cross validation")
    print(rmsle_cv(gpr_model,X,y))
    # joblib.dump(gpr_model, f'models/{sport}_gpr_model.pkl')

c:\Users\PC\.conda\envs\Env\lib\site-packages\sklearn\gaussian_process\_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\PC\.conda\envs\Env\lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\PC\.conda\envs\Env\lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


[-0.00390625 -0.00390625  1.09521484 ...  1.09521484 -0.00390625
 -0.00390625] [3.67427707e-07 3.67427707e-07 5.60727350e-07 ... 5.60727350e-07
 3.67427707e-07 3.67427707e-07]


c:\Users\PC\.conda\envs\Env\lib\site-packages\sklearn\gaussian_process\_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


KeyboardInterrupt: 

In [24]:
import joblib

for sport in sports:
    if len(Data[sport]) == 0:
        continue
    simple = np.array(Data[sport])
    X = np.array(simple[:,0:3])
    y = np.array(simple[:,3])
    linear_model = LinearModel(X,y)
    print(rmsle_cv(linear_model,X,y))
    joblib.dump(linear_model, f'models/{sport}_linear_model.pkl')
    # xgb_model = random_search_params(X,y,model_type='xgb')
    # lgb_model = random_search_params(X,y,model_type='lgb')
    # print(rmsle_cv(xgb_model,X,y))
    # print(rmsle_cv(lgb_model,X,y))
    # joblib.dump(xgb_model, f'models/{sport}_xgb_model.pkl')
    # joblib.dump(lgb_model, f'models/{sport}_lgb_model.pkl')

    gpr_model = GaussianProcessRegressorModel(X,y)
    print(rmsle_cv(gpr_model,X,y))
    joblib.dump(gpr_model, f'models/{sport}_gpr_model.pkl')


    
    
    




截距项 (β0): 0.17535596287273703
回归系数 (β1, β2, β3): [0.00000000e+00 1.11022302e-16 3.63462698e-01]
预测结果: [0.17535596 0.17535596 0.90228136 ... 0.90228136 0.17535596 0.17535596]
[0.08252727 0.13458893 0.14101135 0.10397013 0.12493777]


KeyboardInterrupt: 